In [3]:
import pandas as pd
from tensorflow.python import keras # High level, easy to use, intuitive DNN library
from tensorflow.python.keras.datasets import mnist # Digits data
from tensorflow.python.keras.models import Sequential, Model # Method that helps us build the model,
                                                      # by stacking the layers together
from tensorflow.python.keras.layers import Dense, Dropout, Flatten # Different layers as part of the network;
# Dense layer - is your regular linear layer, # Dropt out sets some of the weights to zero randomly
# to prevent from overfitting; Flatten does exactly what it sounds like, flattens the multi-dimensional array
# into one dimension
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D #2D convolution array, and Max Pooling array
from tensorflow.python.keras import backend as K
import numpy as np
import gc
import tensorflow


In [ ]:
import pandas as pd
obs = pd.read_csv('/modules/cs342/Assignment2/test_set.csv',dtype={'flux':'float32'},usecols=[0,2])
test_meta=pd.read_csv("/modules/cs342/Assignment2/test_set_metadata.csv")

In [ ]:

from time import time
t0=time()
res = obs.assign(ValNum=obs.groupby('object_id').cumcount()+1)\
        .pivot(index='object_id', columns='ValNum', values='flux')\
        .reset_index()
t1=time()
print(t1-t0)

In [17]:
#converts testing set to be entered into conv layer
from time import time
a=0
for obs in pd.read_csv('/modules/cs342/Assignment2/test_set.csv',chunksize=50000000,
                   dtype={'flux':'float32'},usecols=[0,3]):
    t0=time()
    res = obs.assign(ValNum=obs.groupby('object_id').cumcount()+1)\
        .pivot(index='object_id', columns='ValNum', values='flux')\
        .reset_index()
    if a==0:
        #output.to_csv("/var/tmp/kdecache-u1403100/features_1.csv")
        time_data=res
        a=1
        
    else:
        #output.to_csv("/var/tmp/kdecache-u1403100/features_1.csv", mode='a',header=False)
        time_data=pd.concat([time_data,res],axis=0)
       
    t1=time()
    print(t1-t0)
time_data=time_data.drop_duplicates(subset='object_id')

19.3695111275
18.5672860146
17.8629670143
18.1061530113
18.1925430298
18.4580729008
18.4484801292
20.7398569584
20.2759490013
3.44456601143


In [14]:
#converts training set to be entered into conv layer
obs=pd.read_csv('/dcs/17/u1403100/Downloads/A2/training_set.csv',dtype={'flux':'float32'},usecols=[0,3])
train = obs.assign(ValNum=obs.groupby('object_id').cumcount()+1)\
        .pivot(index='object_id', columns='ValNum', values='flux')\
        .reset_index()

In [27]:
time_data=time_data.fillna(0)
train=train.fillna(0)

In [31]:
train_x=train.drop(['object_id'],axis=1).values
train_y=pd.get_dummies(meta['target']).values
test_x=time_data.drop(['object_id'],axis=1).values
train_x = train_x.reshape(7848,352,1)
test_x = test_x.reshape(3492890,352,1)


In [69]:
#Convolution Neural Network from keras cdocumentation

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from keras import optimizers
seq_length = 352
inp =  Input(shape=(352, 1))

model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=(seq_length, 1)))
model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.5))
model.add(Dense(14, activation='sigmoid'))
sgd = optimizers.SGD(lr=0.01, clipnorm=1.)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])


In [70]:
model.fit(train_x, train_y, batch_size=32, nb_epoch=7)


Epoch 1/7
7848/7848 [==============================] - 6s - loss: 2.5847 - acc: 0.3299     
Epoch 2/7
7848/7848 [==============================] - 6s - loss: 2.5602 - acc: 0.3110     
Epoch 3/7
7848/7848 [==============================] - 6s - loss: 2.6186 - acc: 0.3110     
Epoch 4/7
7848/7848 [==============================] - 6s - loss: 2.7250 - acc: 0.3118     
Epoch 5/7
7848/7848 [==============================] - 6s - loss: 2.7163 - acc: 0.3137     
Epoch 6/7
7848/7848 [==============================] - 6s - loss: 2.6805 - acc: 0.3138     
Epoch 7/7
7848/7848 [==============================] - 6s - loss: 2.6669 - acc: 0.3151     


In [71]:
pred=model.predict_proba(test_x)

3492704/3492890 [============================>.] - ETA: 0s

In [72]:
    output = pd.DataFrame({
                        'Class_6':pred[:,0],
                        'Class_15':pred[:,1],
                        'Class_16':pred[:,2],
                        'Class_42':pred[:,3],
                        'Class_52':pred[:,4],
                        'Class_53':pred[:,5],
                        'Class_62':pred[:,6],
                        'Class_64':pred[:,7],
                        'Class_65':pred[:,8],
                        'Class_67':pred[:,9],
                        'Class_88':pred[:,10],
                        'Class_90':pred[:,11],
                        'Class_92':pred[:,12],
                        'Class_95':pred[:,13]                              
                       })
   

In [73]:
test_meta=pd.read_csv("/modules/cs342/Assignment2/test_set_metadata.csv")
output.index = np.arange(0, len(output))
output['object_id']=test_meta['object_id']
output['class_99']=0.06


NameError: name 'ouput' is not defined

In [75]:
output.to_csv("/dcs/17/u1403100/Downloads/A2/Task6.csv",index=False)